<a href="https://colab.research.google.com/github/daniloaleixo/ColabTPUPlayground/blob/master/Simple_Sentiment_Classification_with_TPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simple Sentiment Classification

## Configs

### Imports

In [0]:
from __future__ import print_function

import tensorflow as tf
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.datasets import imdb
import numpy as np
import os
import pprint


### TPU Config

In [2]:
use_tpu = True #@param {type:"boolean"}

if use_tpu:
    assert 'COLAB_TPU_ADDR' in os.environ, 'Missing TPU; did you request a TPU in Notebook Settings?'

if 'COLAB_TPU_ADDR' in os.environ:
  TF_MASTER = 'grpc://{}'.format(os.environ['COLAB_TPU_ADDR'])
else:
  TF_MASTER=''

with tf.Session(TF_MASTER) as session:
  print ('List of devices:')
  pprint.pprint(session.list_devices())

List of devices:
[_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, 4061313884233958147),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 12842224579386048533),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 9486488098452943182),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 7081663475015790559),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 15630161193405178680),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:3, TPU, 17179869184, 11870914052132360289),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:4, TPU, 17179869184, 7089444132808245677),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:5, TPU, 17179869184, 6702658911622933483),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:6, TPU, 17179869184, 17345802987333940722),
 _DeviceAttributes(/job:tp

### Hyperparameters

In [0]:
max_features = 20000
# cut texts after this number of words (among top max_features most common words)
maxlen = 80
batch_size = 32

# TPU address
tpu_address = TF_MASTER

# Number of epochs
epochs = 15

# Number of steps_per_epoch
steps_per_epoch = 5

## Getting data

### Download the data


In [4]:
print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

# Our vectorized labels
y_train = np.asarray(y_train).astype('float32').reshape((-1,1))
y_test = np.asarray(y_test).astype('float32').reshape((-1,1))

Loading data...
25000 train sequences
25000 test sequences


In [0]:
def train_input_fn(batch_size=1024):
  # Convert the inputs to a Dataset.
  dataset = tf.data.Dataset.from_tensor_slices((x_train,y_train))
  # Shuffle, repeat, and batch the examples.
  dataset = dataset.cache()
  dataset = dataset.shuffle(1000, reshuffle_each_iteration=True)
  dataset = dataset.repeat()
  dataset = dataset.batch(batch_size, drop_remainder=True)
  # Return the dataset.
  return dataset

In [0]:
def test_input_fn(batch_size=1024):
  # Convert the inputs to a Dataset.
  dataset = tf.data.Dataset.from_tensor_slices((x_test,y_test))
  # Shuffle, repeat, and batch the examples.
  dataset = dataset.cache()
  dataset = dataset.shuffle(1000, reshuffle_each_iteration=True)
  dataset = dataset.repeat()
  dataset = dataset.batch(batch_size, drop_remainder=True)
  # Return the dataset.
  return dataset

### Padding the data

In [7]:
print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

Pad sequences (samples x time)
x_train shape: (25000, 80)
x_test shape: (25000, 80)


## Building the model

### Model Architecture

In [8]:
print('Build model...')

def get_model():
  model = Sequential()
  model.add(Embedding(max_features, 128))
  model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
  model.add(Dense(1, activation='sigmoid'))
  return model

Build model...


### Compiling the model

In [9]:
print(use_tpu, "use_tpu")
if use_tpu:
  resolver = tf.contrib.cluster_resolver.TPUClusterResolver(TF_MASTER)
  tf.contrib.distribute.initialize_tpu_system(resolver)
  strategy = tf.contrib.distribute.TPUStrategy(resolver)
  with strategy.scope():
    # Optimizer = tf.keras.optimizers.SGD(lr=1e-3, momentum=0.3, decay=0, nesterov=False)
    # Optimizer = adam(lr=0.1, decay=1e-6)
    # Optimizer = tf.keras.optimizers.Adagrad(learning_rate=0.1)
    model = get_model()
    model.compile(
      optimizer=tf.train.RMSPropOptimizer(learning_rate=1e-2),
      loss=tf.keras.losses.binary_crossentropy,
      metrics=[tf.keras.metrics.Accuracy()]
    )

else:
  # try using different optimizers and different optimizer configs
  model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


model.summary()
  

True use_tpu
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

INFO:tensorflow:Initializing the TPU system: 10.21.83.18:8470
INFO:tensorflow:Finished initializing TPU system.
INFO:tensorflow:Querying Tensorflow master (grpc://10.21.83.18:8470) for TPU system metadata.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, -1, 4061313884233958147)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 9486488098452

## Training

In [10]:
print(x_train.shape, y_train.shape)

(25000, 80) (25000, 1)


In [12]:
print('Train...')
model.fit(
    train_input_fn(batch_size),
    # batch_size=batch_size,
    epochs=epochs,
    steps_per_epoch=int(x_train.shape[0] / batch_size),
    validation_data=test_input_fn(batch_size)
)

Train...
Epoch 1/15
781/781 [==============================] - 18s 22ms/step - loss: 0.3284 - accuracy: 0.0000e+00
Epoch 2/15
781/781 [==============================] - 16s 21ms/step - loss: 0.2600 - accuracy: 0.0000e+00
Epoch 3/15
781/781 [==============================] - 16s 21ms/step - loss: 0.2245 - accuracy: 0.0000e+00
Epoch 4/15
781/781 [==============================] - 16s 21ms/step - loss: 0.1979 - accuracy: 0.0000e+00
Epoch 5/15
781/781 [==============================] - 16s 21ms/step - loss: 0.1843 - accuracy: 0.0000e+00
Epoch 6/15
781/781 [==============================] - 17s 22ms/step - loss: 0.1702 - accuracy: 0.0000e+00
Epoch 7/15
781/781 [==============================] - 17s 21ms/step - loss: 0.1593 - accuracy: 1.2004e-04
Epoch 8/15
781/781 [==============================] - 17s 22ms/step - loss: 0.1531 - accuracy: 0.0000e+00
Epoch 9/15
781/781 [==============================] - 17s 22ms/step - loss: 0.1485 - accuracy: 0.0000e+00
Epoch 10/15
781/781 [================

## Accuracy

In [13]:
score, acc = model.evaluate(test_input_fn(batch_size),
                            steps=int(x_test.shape[0] / batch_size),)
print('Test score:', score)
print('Test accuracy:', acc)

781/781 [==============================] - 21s 27ms/step
Test score: 0.5036919158471989
Test accuracy: 0.0
